# Clustering of horror movies

Loading data previously cleaned in R

In [2]:
import pandas as pd
import numpy as np

movies = pd.read_csv("horror_movies_cleaned.csv")
movies

,title,plot,director,cast,Action,Adult,Adventure,Animation,Biography,Comedy,...,Music,Musical,Mystery,Reality-TV,Romance,Sci-Fi,Sport,Thriller,War,Western
0,Gut,Family man Tom has seen something he cant forg...,Elias,"Jason Vail, Nicholas Wilder, Sarah Schoofs, Ki...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,Treasure Chest of Horrors II,"Mona Screamalot, along with her crazy family, ...","M Kelley, Shawn C Phillips, Alex Powers","Veronica Ricci, Nicholas Adam Clark, James Cul...",0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,Infidus,"In the suburbs of Rome, two disparate lives wi...",Giulio De Santi,"Bonini Mino, Massimo Caratelli, Maurizio Zaffi...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,In Extremis,Its the end of the world at least their world.,Steve Stone,"David OHara, Isabelle Allen, Neil Pearson, Toy...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Ghostlight,Andrew wins a radio contest to spend one night...,Jeff Ferrell,"Eden Campbell, Lisa Coronado, David A Crellin,...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,Victor Frankenstein,"Told from Igors perspective, we see the troubl...",Paul McGuigan,"Daniel Radcliffe, James McAvoy, Jessica Brown ...",0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3164,The Exorcism of Molly Hartley,Taking place years after The Haunting of Molly...,Steven R Monroe,"Sarah Lind, Devon Sawa, Gina Holden, Peter Mac...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3165,Talon Falls,Four teenagers on a road trip decide to take a...,Joshua Shreve,"Brad Bell, Jordyn Rudolph, Ryan Rudolph, Morga...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3166,BloodRayne: The Third Reich,Rayne fights against the Nazis in Europe durin...,Uwe Boll,"Natassia Malthe, Brendan Fletcher, Michael Par...",1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Tokenization of plots

Stoppord, tokens, lowercasing, möjligen lemmatisering.

In [27]:
# Using things from lab 1
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load("en_core_web_sm", disable=["ner", "tagger"])

def preprocess(text):
    doc = nlp(text)
    
    ret_list = [token.lemma_.lower() for token  in doc if not token.is_stop]
    return list(filter(lambda x: x.isalpha(), ret_list))

movies["token_plot"] = movies.apply(lambda row: preprocess(row["plot"]), axis = 1)
movies["token_title_plot"] = movies.apply(lambda row: preprocess(row["title"]+" "+row["plot"]), axis = 1)

In [18]:
# Funkar inte just nu, måste ha både splittad lista och mening lagrad nånstans...
vectorizer_p = TfidfVectorizer(stop_words = 'english')
plots_tfidf = vectorizer_p.fit_transform(" ".join(movies["token_plot"]))
vectorizer_tp = TfidfVectorizer(stop_words = 'english')
title_plots_tfidf = vectorizer_tp.fit_transform(" ".join(movies["token_title_plot"]))

TypeError: sequence item 0: expected str instance, list found

## K-means-clustering

### Tuning number of clusters

In [6]:
from sklearn.cluster import KMeans

def inertia(model, data):
    dists = model.transform(data)
    min_dist = np.amin(dists, axis = 1)
    return sum(np.square(min_dist))


max_clus = 35

plot_models = [KMeans(n_clusters=nclus, n_init=6, n_jobs=-1, random_state = max_clus).fit(plots_tfidf) for nclus in range (1,max_clus)]
inertias = [inertia(model, plots_tfidf) for model in plot_models]

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.scatter(range(1,max_clus), inertias)
plt.plot(range(1,max_clus), inertias)
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

Elbow som i labben...

## DBSCAN-clustering

### Tuning eps and min.pts

Ska väl gå på nåt jäkla vis...

## Topic modelling

In [28]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(movies["token_plot"])

corpus = [dictionary.doc2bow(d) for d in movies["token_plot"]]

In [29]:
from gensim.models import LdaModel
temp = dictionary[0]
id2word = dictionary.id2token
model = LdaModel(corpus, num_topics=5, id2word = id2word)

In [30]:
topics = model.top_topics(corpus)
topics

[([(0.0140173985, 'a'),
   (0.011329731, 'young'),
   (0.00963847, 'the'),
   (0.008983575, 'find'),
   (0.008729347, 'friend'),
   (0.008037387, 'house'),
   (0.00605482, 'girl'),
   (0.0060539357, 'home'),
   (0.005926356, 'man'),
   (0.0058904346, 'family'),
   (0.0049904333, 'night'),
   (0.0048435577, 'when'),
   (0.004791152, 'horror'),
   (0.004618592, 'live'),
   (0.0046130675, 'story'),
   (0.0041524596, 'killer'),
   (0.0039796927, 'couple'),
   (0.0039600916, 'haunt'),
   (0.0038405473, 'year'),
   (0.0036781458, 'new')],
  -2.511697362905879),
 ([(0.012766024, 'a'),
   (0.0087540075, 'the'),
   (0.0071693985, 'girl'),
   (0.0066313543, 'group'),
   (0.0064752237, 'fight'),
   (0.0063194176, 'live'),
   (0.005616653, 'when'),
   (0.0052790637, 'town'),
   (0.0047019017, 'new'),
   (0.0042840606, 'turn'),
   (0.0041403486, 'find'),
   (0.004099053, 'young'),
   (0.0040532486, 'zombie'),
   (0.0040505333, 'friend'),
   (0.003876456, 'man'),
   (0.0038376576, 'death'),
   (0.00

### Evaluation

Dot-product av klustertillhörogheter?! Borde ge höga värden om de har lika klusterprobs, men kanske blir problem om man har nonsenskluster.
mappa topics till kategorier och jämföra med sanningen?